In [ ]:
# instal libraries
!pip install numpy pandas openai langchain transformers requests pulp

In [ ]:
# Importing required libraries
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation
import openai  # For AI model interactions
import requests  # For making API requests
import pulp
import warnings
from transformers.pipelines import pipeline  # For NLP model processing

warnings.filterwarnings('ignore')
print("Environment setup complete! All dependencies are installed and ready to use.")

Environment setup complete! All dependencies are installed and ready to use.


In [ ]:
# optimization function
def optimizatio_executor(pork_price, wheat_price, starch_price):
  # Instantiate our problem class
  model = pulp.LpProblem("Cost minimising blending problem", pulp.LpMinimize)
  # Construct our decision variable lists
  sausage_types = ['economy', 'premium']
  ingredients = ['pork', 'wheat', 'starch']
  ing_weight = pulp.LpVariable.dicts("weight kg",
                                     ((i, j) for i in sausage_types for j in ingredients),
                                     lowBound=0,
                                     cat='Continuous')
  # Objective Function
  model += (pulp.lpSum([pork_price * ing_weight[(i, 'pork')] + wheat_price * ing_weight[(i, 'wheat')] + starch_price * ing_weight[(i, 'starch')] for i in sausage_types]))

  # Constraints
  # 350 economy and 500 premium sausages at 0.05 kg
  model += pulp.lpSum([ing_weight['economy', j] for j in ingredients]) == 350 * 0.05
  model += pulp.lpSum([ing_weight['premium', j] for j in ingredients]) == 500 * 0.05

  # Economy has >= 40% pork, premium >= 60% pork
  model += ing_weight['economy', 'pork'] >= (
      0.4 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

  model += ing_weight['premium', 'pork'] >= (
      0.6 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

  # Sausages must be <= 25% starch
  model += ing_weight['economy', 'starch'] <= (
      0.25 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

  model += ing_weight['premium', 'starch'] <= (
      0.25 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

  # We have at most 30 kg of pork, 20 kg of wheat and 17 kg of starch available
  model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) <= 30
  model += pulp.lpSum([ing_weight[i, 'wheat'] for i in sausage_types]) <= 20
  model += pulp.lpSum([ing_weight[i, 'starch'] for i in sausage_types]) <= 17

  # We have at least 23 kg of pork to use up
  model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) >= 23

  # Solve our problem
  model.solve()

  # print output

  if (pulp.LpStatus[model.status] == "Optimal"):

    print("\n\nGreat News! With the input a definite solution has been found. The output are as below\n")

    for var in ing_weight:
      var_value = ing_weight[var].varValue
      print ("The weight of {0} in {1} sausages is {2} kg".format(var[1], var[0], var_value))

    total_cost = pulp.value(model.objective)

    print("The total cost is ${} for 350 economy sausages and 500 premium sausages".format(round(total_cost, 2)))

  else:
    print("I am sorry, but with your input combinations, no feasible solution is found. Would you like to try different combination?")


In [ ]:
# optimizatio_executor(4.1, 2.2, 1.5)

## Create Chat Agent to call the optimization

In [ ]:
# Defining a function to process user input and return an appropriate response
def response_call(user_input):
  responses = {
      "hello": "Hi there! How can I assist you?",
      "rerun": optimization_call,
      "optimize" : "Sure, I can"
  }
  # Returning a response based on user input
  if(user_input.lower() == "rerun"):
    return responses.get(user_input)()
  else:
    return responses.get(user_input, "I'm not sure about that, but I'm learning!")
response_call("hello")

# optimization input call
def optimization_call():
  print("I can help with that. Can you please provide input prices?\n")
  pork_price = float(input("Pork price: "))
  wheat_price = float(input("Wheat price: "))
  starch_price = float(input("Starch price: "))
  optimizatio_executor(pork_price, wheat_price, starch_price)
  return "\n\nOptimization complete"

In [ ]:
# Defining a function to simulate a chatbot interaction
def chat():
    print("Chatbot: Hello! Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")  # Taking user input
        if user_input.lower() == "exit":  # Checking if the user wants to exit
            print("Chatbot: Goodbye!")
            break
        print("Chatbot:", response_call(user_input))  # Processing and returning response

# Example: Running the chatbot interaction
chat()  # Uncomment to test

Chatbot: Hello! Type 'exit' to end the chat.
You: hello
Chatbot: Hi there! How can I assist you?
You: optimize
Chatbot: Sure, I can
You: rerun
I can help with that. Can you please provide input prices?

Pork price: 6.5
Wheat price: 7.8
Starch price: 9.7


Great News! With the input a definite solution has been found. The output are as below

The weight of pork in economy sausages is 7.0 kg
The weight of wheat in economy sausages is 10.5 kg
The weight of starch in economy sausages is 0.0 kg
The weight of pork in premium sausages is 23.0 kg
The weight of wheat in premium sausages is 2.0 kg
The weight of starch in premium sausages is 0.0 kg
The total cost is $292.5 for 350 economy sausages and 500 premium sausages
Chatbot: 

Optimization complete
You: exit
Chatbot: Goodbye!


# **Optimization Problem**


---


We're going to make some sausages!

We have the following ingredients available to us:

Ingredient	Cost (€/kg)	Availability (kg)
Pork	4.32	30
Wheat	2.46	20
Starch	1.86	17
We'll make 2 types of sausage:

Economy (>40% Pork)
Premium (>60% Pork)
One sausage is 50 grams (0.05 kg)

According to government regulations, the most starch we can use in our sausages is 25%

We have a contract with a butcher, and have already purchased 23 kg pork, that will go bad if it's not used.

We have a demand for 350 economy sausages and 500 premium sausages.

We need to figure out how to most cost effectively blend our sausages.

Let's model our problem

pe = Pork in the economy sausages (kg)
we = Wheat in the economy sausages (kg)
se = Starch in the economy sausages (kg)
pp = Pork in the premium sausages (kg)
wp = Wheat in the premium sausages (kg)
sp = Starch in the premium sausages (kg)

We want to minimise costs such that:

Cost = 0.72(pe + pp) + 0.41(we + wp) + 0.31(se + sp)

With the following constraints:
pe + we + se = 350 * 0.05
pp + wp + sp = 500 * 0.05
pe ≥ 0.4(pe + we + se)
pp ≥ 0.6(pp + wp + sp)
se ≤ 0.25(pe + we + se)
sp ≤ 0.25(pp + wp + sp)
pe + pp ≤ 30
we + wp ≤ 20
se + sp ≤ 17
pe + pp ≥ 23